In [2]:
import pprint

In [3]:
class Car:
    def __init__(self, name, fuel, coordinate, orientation):
        
        self.name = name
        self.fuel = fuel
        self.coordinate = coordinate # list of coordinates that the car present in board
        self.orientation = orientation
    
    def carInfo(self):
        print("Name:",self.name , ", Fuel:", self.fuel,", coordinate:",self.coordinate,", orientation:",self.orientation)
        


In [4]:
class Board:

    def __init__(self, line):
        
        self.board = []
        self.cars = [] # list of car objects present in the board
        d = 6 # dimension
        self.line = line
    
        carLetters =   ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N','O','P','Q','R','S','T','U','V','W','X','Y','Z']

        # empty board
        myMatrix = [[0 for x in range(d)] for x in range(d)] 


        # making boards (36 characters)
        a = 0    
        for i in range(6):
            for j in range(6):
                myMatrix[i][j] = line[a]
                a = a+1
        
        # assign board to object
        self.board = myMatrix
        
        
        
        # coordinate of cars
        coordinateDict = {}
        for each in carLetters:
            if each in line: # if car exist in board
                coordinateDict[each] = [[x, y] for x, li in enumerate(self.board) for y, val in enumerate(li) if val==each]

                
        # fuel
        fuelDict = {}
        fuelInfoStr = line[36:].strip()
        if fuelInfoStr != "": # additional info exist
            fuels = fuelInfoStr.split()
            for each in fuels:
                fuelDict[each[0]] = int(each[1:])
            
            
        
        # make car object
        for carName in coordinateDict:
            
            # orientation
            if coordinateDict[carName][0][0] == coordinateDict[carName][1][0]:
                orientation = "horizontal"
            else:
                orientation = "vertical"
            
            if carName in fuelDict:
                self.cars.append(Car(carName, fuelDict[carName], coordinateDict[carName],orientation))
        
            else:
                self.cars.append(Car(carName, 100, coordinateDict[carName],orientation))
                
        


    # take one line (=one board)
    def printBoard(self):
        a = 0
        for i in range(6):
            print(self.line[a:(a+6)]," ")
            a += 6
        
    
    
    def successors(self):
            s = {} # {key:value = carName : dictionary of coordinate that this car can move to(=nextDict)}

            for car in self.cars:
                nextDict ={} # {key:value = "direction": nextList }, direction can be "Left","Right" for horizontal, and "Up","Down" for vertical
                nextList = [] # list of coordinates in one direction 

                # horizontal --
                if car.orientation == "horizontal":
                    #check left
                    left = car.coordinate[0][:] # copy most left coordinate of this car

                    while(left[1] > 0): # until reaches edges of board
                        left[1] -= 1 # check one cell to left

                        if self.board[left[0]][left[1]] == '.': # if that cell is empty
                            nextList.append(left[:]) # append to nextList
                        else:
                            break

                    if nextList: # if not empty
                        nextDict["Left"] = nextList # append to nextDict
                        nextList = []


                    # check right
                    right = car.coordinate[-1][:]

                    while(right[1]<5):
                        right[1] += 1

                        if self.board[right[0]][right[1]] == '.':
                            nextList.append(right[:])
                        else:
                            break

                    if nextList:
                        nextDict["Right"] = nextList



                # vertical |
                if car.orientation == "vertical":
                    #check up
                    up = car.coordinate[0][:]
                    while(up[0] > 0):
                        up[0] -= 1
                        if self.board[up[0]][up[1]] == '.':
                            nextList.append(up[:])

                        else:
                            break
                    if nextList:
                        nextDict["Up"] = nextList
                        nextList = []



                    #check right
                    down = car.coordinate[-1][:]
                    while(down[0]<5):
                        down[0] += 1
                        if self.board[down[0]][down[1]] == '.':
                            nextList.append(down[:])

                        else:
                            break
                    if nextList:
                        nextDict["Down"] = nextList



                # append to successor dictionary
                if nextDict:
                    s[car.name] = nextDict

            return s
                     
                                     
                                


In [5]:
file_path = "SampleInputOutput/Sample/sample-input.txt"


def readPuzzles(file_path):
    hashtag = "#"
    with open(file_path) as file:
        puzzles = [line.rstrip() for line in file]
        puzzles = list(filter(None, puzzles))
        
        for line in puzzles.copy():
            if hashtag in line:
                puzzles.remove(line)
    return puzzles  


In [6]:
def printAllInfo(file_path):
    a=1
    lines = readPuzzles(file_path) # extract lines from input file
    
    for each in lines: # for each board, print infos
        print("--Board ",a,"--")
        test = Board(each)
        test.printBoard()
        print("\n>>Successors: ")
        pprint.pprint(test.successors())
        print("\n>>Car info")
        for car in test.cars:
            car.carInfo()
        print("==================================\n")
        a+=1

In [7]:
printAllInfo(file_path)

--Board  1 --
BBIJ..  
..IJCC  
..IAAM  
GDDK.M  
GH.KL.  
GHFFL.  

>>Successors: 
{'G': {'Up': [[2, 0], [1, 0]]},
 'L': {'Up': [[3, 4]]},
 'M': {'Down': [[4, 5], [5, 5]]}}

>>Car info
Name: A , Fuel: 100 , coordinate: [[2, 3], [2, 4]] , orientation: horizontal
Name: B , Fuel: 100 , coordinate: [[0, 0], [0, 1]] , orientation: horizontal
Name: C , Fuel: 100 , coordinate: [[1, 4], [1, 5]] , orientation: horizontal
Name: D , Fuel: 100 , coordinate: [[3, 1], [3, 2]] , orientation: horizontal
Name: F , Fuel: 100 , coordinate: [[5, 2], [5, 3]] , orientation: horizontal
Name: G , Fuel: 100 , coordinate: [[3, 0], [4, 0], [5, 0]] , orientation: vertical
Name: H , Fuel: 100 , coordinate: [[4, 1], [5, 1]] , orientation: vertical
Name: I , Fuel: 100 , coordinate: [[0, 2], [1, 2], [2, 2]] , orientation: vertical
Name: J , Fuel: 100 , coordinate: [[0, 3], [1, 3]] , orientation: vertical
Name: K , Fuel: 100 , coordinate: [[3, 3], [4, 3]] , orientation: vertical
Name: L , Fuel: 100 , coordinate: [[4,